# Progress file for NER training

## [Installatie](#Installatie)

### Installeer spacy in jupyter notebook env

In [ ]:
import sys
!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install unicode

### Installeer spacy voor windows

1. Installeer anaconda
    <div>Dit geeft een goede python omgeving om in te werken.</div>


2. Installeer spacy in de anaconda terminal
<div class="alert alert-block alert-info">conda install -c conda-forge spacy </div>

### Installeer spacy for linux

In een terminal met python geinstalleerd
<div class="alert alert-block alert-info">pip install spacy && pip install unicode</div>

## [Extracting_data_pickle_naar_csv](#Extracting_data_pickle_naar_csv)

Dit script doet hetvolgende:
1. Dit script start met het kiezen van een pickle file.
2. Colommen **address_1, address_2, person_ctry_code** apart nemen
3. Drop all NAN records
4. Sorteren op **person_ctry_code** en enkel EU landen
5. **address_1 en address_2** in een csv file extracten

<div class="alert alert-block alert-warning">Het gedeelte in commentaar doet het zelfde maar dan met <b>street, city, zip_code.</b> Hiervoor hebben we ondervonden dat er minder ingevulde records zijn.</div>
Windows
<div class="alert alert-block alert-info">python generate-spacy-format.py</div>
Linux
<div class="alert alert-block alert-info">python3 generate-spacy-format.py</div>


In [ ]:
import pickle
import pandas as pd

df = pd.read_pickle("./data/samples/500ksample.pkl")

# extract_street = df.filter(items=['street', 'city', 'zip_code', 'person_ctry_code'])
# dropstreet = extract_street.dropna()
# filterstreet = dropstreet.sort_values(by=[('person_ctry_code')])
# nostreet = filterstreet[filterstreet['person_ctry_code'].isin(['AT','BE','BG','CY','CZ','DE','DK','EE','ES','FI','FR','GR','HR','HU','IE','IT','LT','LU','LV','MT','NL','PO','PT','RO','SE','SI','SK'])]
# dropstreet = dropstreet.sort_values(by=[('person_ctry_code')])
# newstreet = dropstreet[dropstreet['person_ctry_code'].isin(['AT','BE','BG','CY','CZ','DE','DK','EE','ES','FI','FR','GR','HR','HU','IE','IT','LT','LU','LV','MT','NL','PO','PT','RO','SE','SI','SK'])]]
# pf = pf.groupby(['person_ctry_code'])

pf = df.filter(items=['address_1', 'address_2', 'person_ctry_code'])
pf = pf.dropna()
pf = pf.sort_values(by=[('person_ctry_code')])
pf = pf[pf['person_ctry_code'].isin(['AT','BE','BG','CY','CZ','DE','DK','EE','ES','FI','FR','GR','HR','HU','IE','IT','LT','LU','LV','MT','NL','PO','PT','RO','SE','SI','SK'])]
#pf = pf[pf['address_1'].str.match('[0-9]+( )?[a-zA-Z]+|[a-zA-Z]+( )?[0-9]+')]
result = pf.filter(items=['address_1', 'address_2'])

result.to_csv('./data/samples/500ksample-europefilter-address.csv', index=False)

## [Genereer_trainingdata](#Genereer_trainingdata)

## Parsing a csv file with European addresses to spaCy readable format
Dit script gaat ons trainings data bezorgen van een op voorhand gegenereerde csv file.
Doormiddel van gebruik van regex
> TODO: regex uitleggen
<div>
Dit script maakt een "training_EU.py" file in de map /data/trainingData
</div>
<div>
De data komt in het formaar:
</div>

<blockquote>
TRAININGS_DATA = [<br>
    &emsp;("Heipelweg 7,A-8700 Leoben",[(0,9, "STREET"),(10,11, "NUMBER"),(19,25, "CITY"),(12,18, "ZIPCODE")]),<br>
    &emsp;...<br>
    ]<br>
</blockquote>

<div class="alert alert-block alert-warning">Amerika is nog niet volledig klaar hier gaat u nog geen resultaat voor kunnen krijgen</div>

#### Windows
Europa:
<div class="alert alert-block alert-info">python generate-spacy-format.py -Europe <br> python generate-spacy-format.py -e </div>
Amerika:
<div class="alert alert-block alert-info">python generate-spacy-format.py -American <br> python generate-spacy-format.py -a </div>

#### Linux
Europa:
<div class="alert alert-block alert-info">python3 generate-spacy-format.py -Europe <br> python3 generate-spacy-format.py -e </div>
Amerika:
<div class="alert alert-block alert-info">python3 generate-spacy-format.py -American <br> python3 generate-spacy-format.py -a</div>


In [ ]:
import csv
import re
from unidecode import unidecode
with open('./data/samples/500ksample-europefilter-address.csv', newline="\n") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    counter = 0 
    f = open("./output/EU/fouten.txt", 'w')
    f.close
    f = open("./data/trainingData/training_EU.py", 'w')
    f.write("TRAININGS_DATA = [\n")
    f.close()
    counter = 0
    fail_numberstreet = 0
    fail_cityzip = 0 
    for row in csvreader:
        correct = True
        counter += 1
        # print("counter: " + str(counter))
        adres = ','.join(row)
        adres = unidecode(adres)
        if not re.match(".*,.*,.*", adres):
            # print("counter: " + str(counter))
            adreslengte = len(adres)
            adres1 = row[0]
            totale_lengte_straat = len(adres1) + 1
            adres2 = row[1]
            # nummer + straat
            if re.match('^([0-9\-]+( ?bis)?) ([ \u00C0-\u017Fa-zA-Z\'\-]+)', adres1):
                split = re.search('^([0-9\-]+( ?bis)?) ([ \u00C0-\u017Fa-zA-Z\'\-]+)', adres1)
                straat = split.span(3)
                nummer = split.span(1)

            # straat + nummer
            elif re.match('^([ \u00C0-\u017Fa-zA-Z\'\-]+) ([0-9\/\-]+( ?[a-zA-Z]*)?)', adres1):
                split = re.search('^([ \u00C0-\u017Fa-zA-Z\'\-]+) ([0-9\/\-]+( ?[a-zA-Z]*)?)', adres1)
                straat = split.span(1)
                nummer = split.span(2)
            else:
                f = open("./output/EU/fouten.txt", 'a')
                f.write("street: " + adres + "\n")
                f.close()
                fail_numberstreet +=1
                correct = False
            
            # zipcode + city
            if re.match('([ A-Z-]*[0-9]+( ?[A-Z]{2})?) ?([\u00C0-\u0338a-zA-Z ./-]+)$', adres2):
                split = re.search('([ A-Z-]*[0-9]+( ?[A-Z]{2})?) ?([\u00C0-\u0338a-zA-Z ./-]+)$', adres2)
                zipcode = list(split.span(1))
                city = list(split.span(3))
                zipcode[0] += totale_lengte_straat
                zipcode[1] += totale_lengte_straat
                city[0] += totale_lengte_straat
                city[1] += totale_lengte_straat

            # city + zipcode 
            elif re.match('([\u00C0-\u017Fa-zA-Z ./-]+) ?([ A-Z-]*[0-9]+)$', adres2):
                split = re.search('([\u00C0-\u017Fa-zA-Z ./-]+) ?([ A-Z-]*[0-9]+)$', adres2)
                zipcode = list(split.span(2))
                city = list(split.span(1))
                zipcode[0] += totale_lengte_straat
                zipcode[1] += totale_lengte_straat
                city[0] += totale_lengte_straat
                city[1] += totale_lengte_straat
            else:
                f = open("./output/EU/fouten.txt", 'a')
                f.write("city: " + adres + "\n")
                f.close()
                fail_cityzip += 1
                correct = False

            if correct:
                f = open("./data/trainingData/training_EU.py", 'a')
                f.write("\t(\""+adres+"\","+"[("+str(straat[0])+","+str(straat[1])+", \"STREET\"),("+str(nummer[0])+","+str(nummer[1])+", \"NUMBER\"),("+str(city[0])+","+str(city[1])+", \"CITY\"),("+str(zipcode[0])+","+str(zipcode[1])+", \"ZIPCODE\")]),\n")
                f.close()
            # print("(\""+adres+"\","+"[("+str(straatposbegin)+","+str(straatposeind)+", \"STREET\"),("+str(nummerposbegin)+","+str(nummerposeind)+", \"NUMBER\"),("+str(cityposbegin)+","+str(cityposeind)+", \"CITY\"),("+str(zipcodeposbegin)+","+str(zipcodeposeind)+", \"ZIPCODE\")])")
    f = open("./data/trainingData/training_EU.py", "a")
    f.write("]")
    f.close()
    print("failed on street: ", fail_numberstreet)
    print("failed on city: ", fail_cityzip)

## [Prepare_binarydata](#Prepare_binarydata)

Hier wordt de "/data/trainingData/training_EU.py" file omgezet naar binaire data. <br>
Dit wordt dan in de map "/data/binaryData/" als spacy file aangemaakt. De "train.spacy" is de data dat gebruikt wordt als training. Dit is 80% van de totale gegenereerde trainingsdata <br>
"dev.spacy" is de validatiedata dat gebruikt wordt bij het trainen van het model. Dit is de resterende 20% van de totale trainingsdata.

### PrepareData.py

In [ ]:
import spacy
import sys
from spacy.tokens import DocBin
sys.path.insert(0, './data/trainingData')
from training_EU import TRAININGS_DATA as TD

f = open("./output/problems.txt", 'w')
f.close()


nlp = spacy.blank("en")

infixes = nlp.Defaults.infixes + [r'\,']
nlp.tokenizer.infix_finditer = spacy.util.compile_infix_regex(infixes).finditer

db = DocBin()
for text, annotations in TD:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        if span is None:
            f = open("./output/problems.txt", 'a')
            f.write("Problem:" + doc.text + " (" + doc.text[start:end] + ")\n")
            f.close()
        else:
            ents.append(span)
        doc.ents = ents
        db.add(doc)
db.to_disk("./data/binaryData/train.spacy")

db = DocBin()
for text, annotations in TD:
    doc = nlp(text)
    ents = []
    for start, end, label in annotations:
        span = doc.char_span(start, end, label=label)
        if span is not None:
            ents.append(span)
        doc.ents = ents
        db.add(doc)
db.to_disk("./data/binaryData/dev.spacy")

## [Start_training](#Start_training)

Hier komt het moment waar we kunnen beginnen met trainen van het model. Dit gebeurt via de command line. <br>
De spacy train geeft aan dat we de training starten. Vervolgens wordt de config file meegegeven. Bij het "--output" argument staat de locatie waar we het model opslaan. In deze map komt het best gevonden model en het laatste aangemaakte model.<br>
Tot slot geven we onze custom Tokenizer mee deze zorgt ervoor dat we steeds splitsen bij een comma, hiermee krijgen we de comma als een aparte token. Dit verhelpt het probleem dat het model commagetallen gaat lezen.
> Bijvoorbeeld:  "Am Kirchwald 347,6100 Seefeld" => "Am"; "Kirchwald"; "347"; ","; "6100"; "Seefeld"

#### Windows
<div class="alert alert-block alert-info">python -m spacy train ./config/config.cfg --output ./model --code ./config/CustomTokenizer.py</div>

#### Linux
<div class="alert alert-block alert-info">python3 -m spacy train ./config/config.cfg --output ./model --code ./config/CustomTokenizer.py</div>

### Run training

In [ ]:
import os
os.system("python3 -m spacy train ./config/config.cfg --output ./model --code ./config/CustomTokenizer.py")

## [Visualiseer_model](#Visualiseer_model)

In volgende code kan u een visueel beeld krijgen van uitkomsten die het model heeft gevonden.<br>
Dit kan gedaan worden door een csv file mee te geven. Of door zelf adressen in te geven in de list die momenteel in commentaar staat. <br>
Erna wordt het model gekozen hier hebt u de keuze uit het beste model of het laatste model.


#### Windows
<div class="alert alert-block alert-info">python TestModel.py</div>

#### Linux
<div class="alert alert-block alert-info">python3 TestModel.py</div>

<div class="alert alert-block alert-warning">U kan het resultaat bekijken door naar de link <a href="http://localhost:5000">http://localhost:5000</a> te gaan</div>

In [ ]:
import csv
import spacy
from spacy.tokens import DocBin
from spacy import displacy

# test_text = ["Keilalahdentie 4,02150 Espoo", "Weisshsusstrasse 2,52066 Aachen", "14 rue Royale,75008 Paris", "Hüninger Strasse 25,14195 Berlin", "19 bis rue Hoche,49100 Angers"]
test_text = []
with open('./data/samples/500ksample-europefilter-address.csv', newline="\n") as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for row in csvreader:
        row = ','.join(row)
        test_text.append(row)

nlp = spacy.load("./model/model-best")
# nlp = spacy.load("./model/model-last")
list = []
correct_counter = 0
total_counter = 0
for i in test_text:    
    doc = nlp(i)
    if len(doc.ents) == 4:
        total_counter += 1
        if len(i) == len(str(doc.ents[0]) + str(doc.ents[1]) + str(doc.ents[2]) + str(doc.ents[3])) + 3:
            list.append(doc)
            correct_counter += 1
colors = {"STREET": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", "NUMBER": "linear-gradient(90deg, #3f5efb, #fc466b)", "ZIPCODE": "linear-gradient(90deg, #090979, #00d4ff)", "CITY": "linear-gradient(90deg, #eeaeca, #94bbe9)", "OTHER": "linear-gradient(90deg, #22c1c3, #fdbb2d)",}
options = {"ents": ["CITY", "STREET", "NUMBER", "ZIPCODE", "OTHER"], "colors": colors}
print("correctness", correct_counter/total_counter)
print("total", total_counter)
displacy.serve(list, style="ent", options=options)